In [1]:
import pandas as pd
df = pd.read_csv(r'D:\tweets-dates.csv', delimiter=',', skiprows=0, lineterminator='\n',skipinitialspace=True)

KeyboardInterrupt: 

In [ ]:
df.head()

,Unnamed: 0,timestamp,text,lang
0,0,2019-05-27 11:49:14+00,È appena uscito un nuovo video! LES CRYPTOMONN...,it
1,1,2019-05-27 11:49:18+00,Cardano: Digitize Currencies; EOS https://t.co...,en
2,2,2019-05-27 11:49:06+00,Another Test tweet that wasn't caught in the s...,en
3,3,2019-05-27 11:49:22+00,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,en
4,4,2019-05-27 11:49:23+00,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,en


In [5]:
df = df.drop(['Unnamed: 0'], axis = 1)

In [6]:
en_df = df[df["lang"] == 'en']
en_df.sample(10, random_state = 5)

,timestamp,text,lang
9055480,2019-07-24 20:52:40+00,2% of Americans Trust Facebook’s Libra More th...,en
9419550,2019-07-28 12:06:19+00,Indonesia 2077: bayar lauk warteg pake address...,en
15426407,2019-10-29 18:48:56+00,@PeterSchiff Old man shouts at Bitcoin... agai...,en
16015017,2019-11-08 21:11:22+00,$NKN. Push! Push! Push! Its Rising! $BTC marke...,en
7306217,2019-07-02 00:00:06+00,#NPXS - Rapid Long Period increase 1h: 12.5% V...,en
7813526,2019-07-09 01:57:45+00,@PeterLBrandt Litecoin is a great price today ...,en
10691211,2019-08-15 20:20:39+00,BUY signal for $WAVES/$BTC[2] on #Binance\nGen...,en
14914658,2019-10-22 08:49:04+00,Is Peter Schiff Warming To Bitcoin With His An...,en
13408290,2019-09-26 13:47:03+00,Emsisoft Releases Bug Fix for Bitcoin-Ransomin...,en
13445635,2019-09-27 00:29:06+00,Block 596765\nHash: 0x...0847ac95393eebe3143f8...,en


In [7]:
en_df.isnull().sum()

timestamp    0
text         0
lang         0
dtype: int64

In [8]:
en_df.dropna(how='any', inplace=True)
en_df.sample(3)

<ipython-input-8-7597484c46c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_df.dropna(how='any', inplace=True)


,timestamp,text,lang
60333,2019-05-11 01:16:07+00,"Buy BTC/USDT at #TokensNet, sell at #Bitfinex;...",en
16293735,2019-11-13 16:51:39+00,"Signup on #DACX Exchange is quick and simple, ...",en
3403514,2017-10-10 09:00:52+00,#Bitcoin Price Watch; Here’s What’s On This Mo...,en


In [9]:
en_df = en_df.reset_index()
en_df.head()

,index,timestamp,text,lang
0,1,2019-05-27 11:49:18+00,Cardano: Digitize Currencies; EOS https://t.co...,en
1,2,2019-05-27 11:49:06+00,Another Test tweet that wasn't caught in the s...,en
2,3,2019-05-27 11:49:22+00,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,en
3,4,2019-05-27 11:49:23+00,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,en
4,6,2019-05-27 11:49:25+00,@nwoodfine We have been building on the real #...,en


In [10]:
en_df = en_df.drop(['index'],axis = 1)

In [11]:
en_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13086448 entries, 0 to 13086447
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  object
 1   text       object
 2   lang       object
dtypes: object(3)
memory usage: 299.5+ MB


In [12]:
import preprocessor as p
from preprocessor.api import clean, tokenize, parse

In [13]:
en_df = en_df.rename(columns = {'text' : 'Tweet'})

In [15]:
from textblob import TextBlob

sentiments=[]
subjectivity=[]
for i in en_df['Tweet']:
    textblb = TextBlob(i)
    sentiments.append(textblb.polarity)
    subjectivity.append(textblb.subjectivity)

In [16]:
pospol=0
negpol=0
nopol=0
subcount=0
for i in sentiments:
        if (i == 0):
            nopol+=1
        elif(i>0):
            pospol+=1
        elif(i<0):
            negpol+=1
for i in subjectivity:
    if(i>0.5):
        subcount+=1
print(' Zero polarities are '+ str(nopol))
print(' Positive polarities are '+ str(pospol))
print(' Negative polarities are '+str(negpol))
print('Number of subjective tweets are : '+ str(subcount))

 Zero polarities are 6570761
 Positive polarities are 5038233
 Negative polarities are 1477454
Number of subjective tweets are : 2912585


In [17]:
en_df['Sentiment']=sentiments
en_df['Subjectivity']=subjectivity

In [18]:
label = []

for i in range(len(en_df)):
    if en_df['Subjectivity'][i] >= 0.5:
        label.append(1)
    else:
        label.append(0)

In [19]:
en_df['Label'] = label

In [20]:
en_df['Label'].value_counts()

0    9411855
1    3674593
Name: Label, dtype: int64

In [22]:
en_df.to_csv(r'C:\Users\SHEEL\Desktop\tweets-eng.csv')